# 31강 비동기 처리

- 동기 처리와 비동기 처리
  - 동기 처리 : 여러 작업을 직렬로 수행, 한번에 하나의 작업만 수행하므로 실행 속도가 느림
  - 비동기 처리 : 여러 작업을 병렬로 수행, 한꺼번에 처리하므로 실행 속도가 빠름

In [2]:
# 프로그램에 딜레이를 적용하는 기능을 제공하는 모듈
import time

# 날짜 처리 모듈
import datetime as dt

# 앞 단원에서 구현한 메일 발송 모듈
import MyMailer

# 비동기 처리 기능을 제공하는 모듈
import concurrent.futures as futures

In [1]:
def timeWork(name, seconds):
    print("[%s] 작업을 %d초 동안 수행합니다." % (name, seconds))
    
    for i in range(0, seconds):
        time.sleep(1)
        print("[%s] %d초..." % (name, i+1))

    print("[%s] 작업이 종료되었습니다." % name)

In [3]:
# 순차적으로 작업이 진행되는 예시
startTime = dt.datetime.now()

timeWork("A", 3)
timeWork("B", 5)
timeWork("C", 2)

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

[A] 작업을 3초 동안 수행합니다.
[A] 1초...
[A] 2초...
[A] 3초...
[A] 작업이 종료되었습니다.
[B] 작업을 5초 동안 수행합니다.
[B] 1초...
[B] 2초...
[B] 3초...
[B] 4초...
[B] 5초...
[B] 작업이 종료되었습니다.
[C] 작업을 2초 동안 수행합니다.
[C] 1초...
[C] 2초...
[C] 작업이 종료되었습니다.
작업에 소요된 시간은 총 10초 입니다.


In [4]:
# 비동기 함수를 사용한 병렬 작업 실행 예시
startTime = dt.datetime.now()

with futures.ThreadPoolExecutor(max_workers = 3) as executor:
    executor.submit(timeWork, "A", 3)
    executor.submit(timeWork, "B", 5)
    executor.submit(timeWork, "C", 2)

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

[A] 작업을 3초 동안 수행합니다.
[B] 작업을 5초 동안 수행합니다.
[C] 작업을 2초 동안 수행합니다.
[A] 1초...
[C] 1초...
[B] 1초...
[A] 2초...
[C] 2초...
[C] 작업이 종료되었습니다.
[B] 2초...
[A] 3초...
[A] 작업이 종료되었습니다.
[B] 3초...
[B] 4초...
[B] 5초...
[B] 작업이 종료되었습니다.
작업에 소요된 시간은 총 5초 입니다.


In [7]:
# 메일링 리스트 개선
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

fromAddr = "yijingue@naver.com"
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

with open('mailtest/content.txt', 'r', encoding = 'utf-8') as f:
    contentTmpl = f.read()

In [11]:
# 기존 순차적 방식의 메일 발송
startTime = dt.datetime.now()

with open("mailtest/mail_list.csv", "r", encoding = 'euc-kr') as f:
    csv = f.readlines()

    for line in csv:
        name, email, file1, file2 = line.strip().split(",")
        toAddr = "{email}".format(email = email)
        subject = subjectTmpl.format(name = name, yy = year, mm = month)
        content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)
       
        MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

작업에 소요된 시간은 총 2초 입니다.


In [17]:
# 비동기식 메일 발송
startTime = dt.datetime.now()

with open("mailtest/mail_list.csv", "r", encoding = 'euc-kr') as f:
    csv = f.readlines()

    with futures.ThreadPoolExecutor(max_workers = 10) as executor:
        for line in csv:
            name, email, file1, file2 = line.strip().split(",")
            toAddr = "{email}".format(email = email)
            subject = subjectTmpl.format(name = name, yy = year, mm = month)
            content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)
        
            executor.submit(MyMailer.sendMail, fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endTime - startTime
print(f"작업에 소요된 시간은 총 {workTime.seconds}초 입니다.")

작업에 소요된 시간은 총 0초 입니다.
